## 특정 영역(독도 반경 100km)의 증감률 분석    
공간 : 주어진 좌표에서 일정 반경    
시간 : 기준 시점 이전/이후      
대상 : 수치형 속성 데이터 (격자별 수온 또는 생물종의 수 등) 

### 필요한 패키지(모듈/라이브러리) 준비

In [2]:
import numpy as np   #배열 등의 수치 데이터 처리용 패키지 
import xarray as xr  #netcdf와 같은 다차원 배열 데이터 처리용 패키지

In [27]:
import pandas as pd  #2차원 표 형태의 데이터프레임 처리용 패키지
import geopandas as gpd  #pandas를 기반으로 벡터 지오메트리(점/선/면)를 포함한 GeoDataFrame을 처리하는 패키지
from shapely.geometry import Point, Polygon #점, 선, 면 Geometry 처리용 패키지 
import plotly.express as px #차트 시각화 패키지 
pd.set_option('float_format', '{:f}'.format)  # 숫자가 지수 형태로 표시되지 않도록 

### 데이터 로딩   
EU Copernicus에서 제공하는 월별 해양 관측 데이터를 활용   
netcdf라는 시공간 다차원 데이터에 수온, 염분 등의 정보가 담겨있음    
https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=MULTIOBS_GLO_PHY_TSUV_3D_MYNRT_015_012  


In [5]:
ds2006 = xr.open_dataset('../data/netcdf/2020/dataset-armor-3d-rep-monthly_20200615T1200Z_P20211122T1135Z.nc')  
ds2006

<xarray.Dataset>
Dimensions:    (time: 1, depth: 50, latitude: 689, longitude: 1440)
Coordinates:
  * time       (time) datetime64[ns] 2020-06-15T12:00:00
  * depth      (depth) int16 0 5 10 15 20 25 ... 3000 3500 4000 4500 5000 5500
  * latitude   (latitude) float32 -82.12 -81.88 -81.62 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
Data variables:
    so         (time, depth, latitude, longitude) float32 ...
    to         (time, depth, latitude, longitude) float32 ...
    ugo        (time, depth, latitude, longitude) float32 ...
    vgo        (time, depth, latitude, longitude) float32 ...
    zo         (time, depth, latitude, longitude) float32 ...
    mlotst     (time, latitude, longitude) float32 ...
Attributes:
    description:  ARMOR3D REP CMEMS December 2020 Release
    title:        ARMOR3D REP - TSHUVMld Global Ocean Observation-based Produ...
    Conventions:  CF-1.0
    institution:  CLS
    domain_name:  GLO
    history:      2020-06-15 00:00:00 ARMOR3D REP - TSHUV Global Ocean Observ...

In [6]:
# 나머지 연월별 데이터들도 로딩 
ds2007 = xr.open_dataset('../data/netcdf/2020/dataset-armor-3d-rep-monthly_20200715T1200Z_P20211122T1136Z.nc')  
ds2008 = xr.open_dataset('../data/netcdf/2020/dataset-armor-3d-rep-monthly_20200815T1200Z_P20211122T1137Z.nc')  
ds2106 = xr.open_dataset('../data/netcdf/2021/dataset-armor-3d-rep-monthly_20210615T1200Z_P20221118T1124Z.nc')  
ds2107 = xr.open_dataset('../data/netcdf/2021/dataset-armor-3d-rep-monthly_20210715T1200Z_P20221118T1126Z.nc')  
ds2108 = xr.open_dataset('../data/netcdf/2021/dataset-armor-3d-rep-monthly_20210815T1200Z_P20221118T1129Z.nc')  

In [8]:
# 한반도 범위의 데이터들을 추출 - # 원하는 경도와 위도 범위 설정
lon_min, lon_max = 123, 133  # 경도 범위
lat_min, lat_max = 32, 43  # 위도 범위

# 해당 범위의 데이터 추출
ds2006 = ds2006.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_min, lat_max))
ds2007 = ds2007.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_min, lat_max))
ds2008 = ds2008.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_min, lat_max))
ds2106 = ds2106.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_min, lat_max))
ds2107 = ds2107.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_min, lat_max))
ds2108 = ds2108.sel(longitude=slice(lon_min, lon_max), latitude=slice(lat_min, lat_max))

In [15]:
# 데이터를 판다스 데이터프레임으로 변환 
df2006 = ds2006.to_dataframe().reset_index()
df2007 = ds2007.to_dataframe().reset_index()
df2008 = ds2008.to_dataframe().reset_index()

df2106 = ds2106.to_dataframe().reset_index()
df2107 = ds2107.to_dataframe().reset_index()
df2108 = ds2108.to_dataframe().reset_index()

In [16]:
# 데이터프레임의 기초 정보 확인
df2006.info()
df2006.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88000 entries, 0 to 87999
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       88000 non-null  datetime64[ns]
 1   depth      88000 non-null  int16         
 2   latitude   88000 non-null  float32       
 3   longitude  88000 non-null  float32       
 4   so         31102 non-null  float32       
 5   to         31102 non-null  float32       
 6   ugo        24891 non-null  float32       
 7   vgo        25063 non-null  float32       
 8   zo         27874 non-null  float32       
 9   mlotst     61400 non-null  float32       
dtypes: datetime64[ns](1), float32(8), int16(1)
memory usage: 3.5 MB


,time,depth,latitude,longitude,so,to,ugo,vgo,zo,mlotst
0,2020-06-15 12:00:00,0,32.125,123.125,31.301001,21.125000,0.054,0.185,0.769,11.099854
1,2020-06-15 12:00:00,0,32.125,123.375,31.400002,20.431000,0.019,0.049,0.793,11.000000
2,2020-06-15 12:00:00,0,32.125,123.625,31.518002,19.997000,0.013,-0.073,0.787,12.800049
3,2020-06-15 12:00:00,0,32.125,123.875,31.604000,19.955000,0.015,-0.125,0.765,10.800049
4,2020-06-15 12:00:00,0,32.125,124.125,31.707001,19.594999,0.031,-0.100,0.741,10.699951


In [19]:
# 2020년 6~8월 3개 데이터를 병합
df20_a = pd.concat([df2006, df2007, df2008], ignore_index=True)
df20_a.info()
df20_a.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264000 entries, 0 to 263999
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   time       264000 non-null  datetime64[ns]
 1   depth      264000 non-null  int16         
 2   latitude   264000 non-null  float32       
 3   longitude  264000 non-null  float32       
 4   so         93306 non-null   float32       
 5   to         93306 non-null   float32       
 6   ugo        74673 non-null   float32       
 7   vgo        75189 non-null   float32       
 8   zo         83622 non-null   float32       
 9   mlotst     184200 non-null  float32       
dtypes: datetime64[ns](1), float32(8), int16(1)
memory usage: 10.6 MB


,time,depth,latitude,longitude,so,to,ugo,vgo,zo,mlotst
0,2020-06-15 12:00:00,0,32.125,123.125,31.301001,21.125000,0.054,0.185,0.769,11.099854
1,2020-06-15 12:00:00,0,32.125,123.375,31.400002,20.431000,0.019,0.049,0.793,11.000000
2,2020-06-15 12:00:00,0,32.125,123.625,31.518002,19.997000,0.013,-0.073,0.787,12.800049
3,2020-06-15 12:00:00,0,32.125,123.875,31.604000,19.955000,0.015,-0.125,0.765,10.800049
4,2020-06-15 12:00:00,0,32.125,124.125,31.707001,19.594999,0.031,-0.100,0.741,10.699951


In [20]:
# 2021년 6~8월 3개 데이터를 병합
df21_a = pd.concat([df2106, df2107, df2108], ignore_index=True)

In [21]:
# 표층(depth = 0) 데이터만 추출
df20_b = df20_a[ df20_a['depth'] == 0   ]
df21_b = df21_a[ df21_a['depth'] == 0   ]

In [22]:
# 20년 데이터의 경도-위도별 평균 수온을 산출
df20_c = df20_b.groupby(['latitude', 'longitude'])['to'].mean().reset_index()
df20_c.info()
df20_c.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   1760 non-null   float32
 1   longitude  1760 non-null   float32
 2   to         1228 non-null   float32
dtypes: float32(3)
memory usage: 20.8 KB


,latitude,longitude,to
0,32.125,123.125,24.171333
1,32.125,123.375,23.760332
2,32.125,123.625,23.768667
3,32.125,123.875,23.738001
4,32.125,124.125,23.573334


In [23]:
# 21년 데이터의 경도-위도별 평균 수온을 산출
df21_c = df21_b.groupby(['latitude', 'longitude'])['to'].mean().reset_index()
df21_c.info()
df21_c.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   1760 non-null   float32
 1   longitude  1760 non-null   float32
 2   to         1228 non-null   float32
dtypes: float32(3)
memory usage: 20.8 KB


,latitude,longitude,to
0,32.125,123.125,25.633667
1,32.125,123.375,25.672998
2,32.125,123.625,25.632334
3,32.125,123.875,25.236000
4,32.125,124.125,25.035334


In [25]:
# 0.25도 간격의 폴리곤 생성 함수
def create_polygon(lat, lon):
    return Polygon([(lon, lat),
                    (lon + 0.25, lat),
                    (lon + 0.25, lat + 0.25),
                    (lon, lat + 0.25)])

In [29]:
# 각 좌표에 대한 폴리곤 생성
geometry20 = df20_c.apply(lambda row: create_polygon(row['latitude'], row['longitude']), axis=1)
# DataFrame을 GeoDataFrame으로 변환
gdf20 = gpd.GeoDataFrame(df20_c, geometry=geometry20, crs='4326')

In [32]:
# GeoDataFrame 기초정보 확인 
gdf20.info()
gdf20.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   latitude   1760 non-null   float32 
 1   longitude  1760 non-null   float32 
 2   to         1228 non-null   float32 
 3   geometry   1760 non-null   geometry
dtypes: float32(3), geometry(1)
memory usage: 34.5 KB


,latitude,longitude,to,geometry
0,32.125,123.125,24.171333,"POLYGON ((123.12500 32.12500, 123.37500 32.125..."
1,32.125,123.375,23.760332,"POLYGON ((123.37500 32.12500, 123.62500 32.125..."
2,32.125,123.625,23.768667,"POLYGON ((123.62500 32.12500, 123.87500 32.125..."
3,32.125,123.875,23.738001,"POLYGON ((123.87500 32.12500, 124.12500 32.125..."
4,32.125,124.125,23.573334,"POLYGON ((124.12500 32.12500, 124.37500 32.125..."


In [56]:
# 2021년 데이터도 공간데이터화 
# 각 좌표에 대한 폴리곤 생성
geometry21 = df21_c.apply(lambda row: create_polygon(row['latitude'], row['longitude']), axis=1)
# DataFrame을 GeoDataFrame으로 변환
gdf21 = gpd.GeoDataFrame(df21_c, geometry=geometry21, crs='4326')

In [67]:
gdf20.explore('to')

In [68]:
gdf21.explore('to')

#### 공간 영역 설정

https://www.openstreetmap.org/ 에 접속해서 '독도'를 검색하고, 울릉군 독도리에 해당하는 독도를 클릭하면 해당 위치로 지도가 표시됨     
지도의 독도 상에서 컨텍스트 메뉴의 "여기를 가운데로 지정"를 선택하면, 독도에 대한 좌표가 URL에 보임     
https://www.openstreetmap.org/relation/6646538#map=16/37.2422/131.8669   

또는 epsg.io.map에 접속해도 쉽게 좌표를 확인할 수 있음    
https://epsg.io/map#srs=4326&x=131.866279&y=37.241672&z=15&layer=streets   
131.866279 37.241672

In [35]:
# 특정 위도, 경도 및 버퍼 거리를 설정합니다.
longitude = 131.866279
latitude = 37.241672
buffer_distance = 100000   #100킬로미터  

# 경위도 Point GeoDataFrame을 생성합니다.
geom_latlon = Point(longitude, latitude)
gdf_pt4326 = gpd.GeoDataFrame(geometry=[geom_latlon], crs='epsg:4326' ) 

# Point GeoDataFrame을 평면직각좌표계(3857)로 변환
gdf_pt3857 = gdf_pt4326.to_crs(epsg=3857) 

# gdf_pt3857에 대한 버퍼 공간데이터를 생성 
geom_buf = gdf_pt3857.geometry.buffer(buffer_distance)

# 버퍼 폴리곤을 GeoDataFrame으로 변환합니다.
gdf_buf3857 = gpd.GeoDataFrame(geometry=geom_buf ) 

# 버퍼 폴리곤을 4326으로 변환 (생물종과 공간조인을 위해)
gdf_buf4326 = gdf_buf3857.to_crs(epsg=4326) 

In [36]:
gdf_buf3857.explore()

In [57]:
# # 연도 구분 컬럼 생성
# gdf20['year'] = 2020
# 위도-경도를 이용한 고유ID 부여
gdf20['lat_lon'] = gdf20['latitude'].astype(str) + '_' + gdf20['longitude'].astype(str)
# 독도 반경 영역에 해당하는 데이터만 추출 
gdf20_b = gpd.sjoin(gdf20, gdf_buf4326, how='inner', predicate='intersects', lsuffix='left' ).reset_index()
gdf20_b.info()
gdf20_b.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   index        47 non-null     int64   
 1   latitude     47 non-null     float32 
 2   longitude    47 non-null     float32 
 3   to           47 non-null     float32 
 4   geometry     47 non-null     geometry
 5   lat_lon      47 non-null     object  
 6   index_right  47 non-null     int64   
dtypes: float32(3), geometry(1), int64(2), object(1)
memory usage: 2.1+ KB


,index,latitude,longitude,to,geometry,lat_lon,index_right
0,713,36.375,131.375,22.417000,"POLYGON ((131.37500 36.37500, 131.62500 36.375...",36.375_131.375,0
1,714,36.375,131.625,22.459000,"POLYGON ((131.62500 36.37500, 131.87500 36.375...",36.375_131.625,0
2,715,36.375,131.875,22.700668,"POLYGON ((131.87500 36.37500, 132.12500 36.375...",36.375_131.875,0
3,716,36.375,132.125,22.816000,"POLYGON ((132.12500 36.37500, 132.37500 36.375...",36.375_132.125,0
4,751,36.625,130.875,22.438334,"POLYGON ((130.87500 36.62500, 131.12500 36.625...",36.625_130.875,0


In [42]:
gdf20_b.explore('to')

In [58]:
# # 연도 구분 컬럼 생성
# gdf21['year'] = 2021
# 위도-경도를 이용한 고유ID 부여
gdf21['lat_lon'] = gdf21['latitude'].astype(str) + '_' + gdf21['longitude'].astype(str)
# 독도 반경 영역에 해당하는 데이터만 추출 
gdf21_b = gpd.sjoin(gdf21, gdf_buf4326, how='inner', predicate='intersects', lsuffix='left' ).reset_index()

In [53]:
gdf21_b.explore('to')

증감율 = [(기준시점 이후의 측정값 - 기준시점 이전의 측정값) ÷ 기준시점 이전의 측정값] × 100    
증가율이 100%면 2배 증가한 것 

In [59]:
# lat_lon 값을 기준으로 데이터 병합
merged_df = gdf20_b.merge(gdf21_b, on='lat_lon', suffixes=('_2020', '_2021'))

In [60]:
merged_df.head()

,index_2020,latitude_2020,longitude_2020,to_2020,geometry_2020,lat_lon,index_right_2020,index_2021,latitude_2021,longitude_2021,to_2021,geometry_2021,index_right_2021
0,713,36.375,131.375,22.417000,"POLYGON ((131.37500 36.37500, 131.62500 36.375...",36.375_131.375,0,713,36.375,131.375,23.767668,"POLYGON ((131.37500 36.37500, 131.62500 36.375...",0
1,714,36.375,131.625,22.459000,"POLYGON ((131.62500 36.37500, 131.87500 36.375...",36.375_131.625,0,714,36.375,131.625,23.817667,"POLYGON ((131.62500 36.37500, 131.87500 36.375...",0
2,715,36.375,131.875,22.700668,"POLYGON ((131.87500 36.37500, 132.12500 36.375...",36.375_131.875,0,715,36.375,131.875,23.858000,"POLYGON ((131.87500 36.37500, 132.12500 36.375...",0
3,716,36.375,132.125,22.816000,"POLYGON ((132.12500 36.37500, 132.37500 36.375...",36.375_132.125,0,716,36.375,132.125,23.991333,"POLYGON ((132.12500 36.37500, 132.37500 36.375...",0
4,751,36.625,130.875,22.438334,"POLYGON ((130.87500 36.62500, 131.12500 36.625...",36.625_130.875,0,751,36.625,130.875,23.951666,"POLYGON ((130.87500 36.62500, 131.12500 36.625...",0


In [61]:
# 증감률 계산: [(2021년 값 - 2020년 값) / 2020년 값] * 100
merged_df['growth_rate'] = ((merged_df['to_2021'] - merged_df['to_2020']) / merged_df['to_2020']) * 100

print(merged_df[['lat_lon', 'growth_rate']])

           lat_lon  growth_rate
0   36.375_131.375     6.025195
1   36.375_131.625     6.049545
2   36.375_131.875     5.098226
3   36.375_132.125     5.151354
4   36.625_130.875     6.744406
5   36.625_131.125     6.840475
6   36.625_131.375     6.971265
7   36.625_131.625     5.965507
8   36.625_131.875     5.217985
9   36.625_132.125     4.368376
10  36.625_132.375     3.233400
11  36.625_132.625     3.556391
12  36.875_130.875     7.070722
13  36.875_131.125     7.321136
14  36.875_131.375     7.203534
15  36.875_131.625     5.809846
16  36.875_131.875     6.064041
17  36.875_132.125     5.240392
18  36.875_132.375     4.639335
19  36.875_132.625     4.248854
20  37.125_130.875     9.561443
21  37.125_131.125     9.419824
22  37.125_131.375     8.360932
23  37.125_131.625     7.745859
24  37.125_131.875     7.043304
25  37.125_132.125     5.901570
26  37.125_132.375     4.901421
27  37.125_132.625     5.025776
28  37.375_130.875     9.079633
29  37.375_131.125     9.227164
30  37.3

In [62]:
# 증감률이 높은 순서대로 정렬
merged_df2 = merged_df.sort_values(by='growth_rate', ascending=False)
merged_df2.head()

,index_2020,latitude_2020,longitude_2020,to_2020,geometry_2020,lat_lon,index_right_2020,index_2021,latitude_2021,longitude_2021,to_2021,geometry_2021,index_right_2021,growth_rate
20,831,37.125,130.875,22.064333,"POLYGON ((130.87500 37.12500, 131.12500 37.125...",37.125_130.875,0,831,37.125,130.875,24.174002,"POLYGON ((130.87500 37.12500, 131.12500 37.125...",0,9.561443
36,911,37.625,130.875,21.951666,"POLYGON ((130.87500 37.62500, 131.12500 37.625...",37.625_130.875,0,911,37.625,130.875,24.032999,"POLYGON ((130.87500 37.62500, 131.12500 37.625...",0,9.481436
21,832,37.125,131.125,22.102333,"POLYGON ((131.12500 37.12500, 131.37500 37.125...",37.125_131.125,0,832,37.125,131.125,24.184334,"POLYGON ((131.12500 37.12500, 131.37500 37.125...",0,9.419824
29,872,37.375,131.125,22.018333,"POLYGON ((131.12500 37.37500, 131.37500 37.375...",37.375_131.125,0,872,37.375,131.125,24.050001,"POLYGON ((131.12500 37.37500, 131.37500 37.375...",0,9.227164
28,871,37.375,130.875,22.064001,"POLYGON ((130.87500 37.37500, 131.12500 37.375...",37.375_130.875,0,871,37.375,130.875,24.067331,"POLYGON ((130.87500 37.37500, 131.12500 37.375...",0,9.079633


In [63]:
# Plotly 막대 차트를 그립니다.
fig = px.bar(merged_df2, x='lat_lon', y='growth_rate', title='2020년 대비 2021년 여름(6~8월) 수온 증감률')

# x축과 y축의 제목을 설정합니다.
fig.update_layout(
    xaxis_title="격자(위도_경도)",
    yaxis_title="증감률",
)

fig.show()

### 지도 시각화

In [64]:
# 증감률 데이터에서 필요한 항목만 추출
merged_df3 = merged_df2[['lat_lon', 'growth_rate']]

In [65]:
# 2021년 공간(격자)데이터와 증감률 데이터를 속성조인
gdf21_c = pd.merge(gdf21_b, merged_df3, on='lat_lon', how='inner')

In [66]:
# 증감률을 지도시각화 
gdf21_c.explore('growth_rate', cmap='Reds', tiles='http://xdworld.vworld.kr:8080/2d/Satellite/service/{z}/{x}/{y}.jpeg', attr='브이월드')